In [1]:
!pip install pandas numpy scikit-learn xgboost tensorflow gradio statsmodels

!pip install --upgrade scikit-learn xgboost
!pip install pandas numpy tensorflow matplotlib
!pip install scikit-learn==1.3.0
!pip install xgboost


  Using cached scikit_learn-1.6.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
Using cached scikit_learn-1.6.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.5 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.0
    Uninstalling scikit-learn-1.3.0:
      Successfully uninstalled scikit-learn-1.3.0
  Using cached scikit_learn-1.3.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
Using cached scikit_learn-1.3.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.9 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imbalanced-learn 0.13.0 requires scikit-learn<2,>=1

In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import gradio as gr

# Load data
file_path = 'europeus.csv'
data = pd.read_csv(file_path, on_bad_lines='skip')

# Handling potential issues in the 'Time' column
try:
    data['Time'] = pd.to_datetime(data['Time'], errors='coerce')  # Handle errors by setting them to NaT
except pd.errors.ParserError as e:
    print(f"Error parsing 'Time' column: {e}")
    # Additional error handling if the 'Time' column format is unpredictable

# Remove rows with NaT (Not a Time) values in the 'Time' column
data = data.dropna(subset=['Time'])

# Set 'Time' as index
data.set_index('Time', inplace=True)

# Feature engineering
data['hour'] = data.index.hour
data['day_of_week'] = data.index.dayofweek
data['month'] = data.index.month
data['Close_Lag_1'] = data['Close'].shift(1)
data['Close_Lag_2'] = data['Close'].shift(2)
data['Close_MA_3'] = data['Close'].rolling(window=3).mean()
data['Close_MA_7'] = data['Close'].rolling(window=7).mean()
data['Close_EMA_3'] = data['Close'].ewm(span=3).mean()
data['Close_EMA_7'] = data['Close'].ewm(span=7).mean()
data = data.dropna()

# Split training and testing data
train_data = data[data.index < '2025-01-01']
test_data = data[data.index >= '2025-01-01']

# Features and target
features = ['Open', 'High', 'Low', 'Volume', 'hour', 'day_of_week', 'month',
            'Close_Lag_1', 'Close_Lag_2', 'Close_MA_3', 'Close_MA_7', 'Close_EMA_3', 'Close_EMA_7']
X_train = train_data[features]
y_train = train_data['Close']

# Scale features for models
scaler_X = StandardScaler()
scaler_y = MinMaxScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))

# XGBoost Model
from sklearn.exceptions import NotFittedError

# Ensure data is clean
if np.isnan(X_train_scaled).sum() > 0 or np.isinf(X_train_scaled).sum() > 0:
    print("Fixing NaNs/Infs in training data...")
    X_train.fillna(X_train.mean(), inplace=True)
    X_train_scaled = scaler_X.fit_transform(X_train)

# Validate hyperparameters
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1]
}

# Initialize XGBoost model with 'tree_method' set to 'hist' for CPU training
xgb_model = XGBRegressor(tree_method='hist')  # Force CPU training using 'hist'

try:
    # Fit XGBoost with RandomizedSearchCV
    xgb_random_search = RandomizedSearchCV(
        xgb_model, param_grid, n_iter=10, cv=3, scoring='neg_mean_squared_error', n_jobs=-1,
        random_state=42, error_score='raise'
    )
    xgb_random_search.fit(X_train_scaled, y_train)

    # Assign the best model
    best_xgb_model = xgb_random_search.best_estimator_

except ValueError as e:
    print(f"Error during model training: {e}")
    raise


# LSTM Model
X_train_lstm = X_train_scaled.reshape(X_train_scaled.shape[0], 1, X_train_scaled.shape[1])

lstm_model = Sequential([
    LSTM(100, activation='relu', input_shape=(1, X_train_scaled.shape[1]), return_sequences=True),
    Dropout(0.2),
    LSTM(50, activation='relu', return_sequences=False),
    Dropout(0.2),
    Dense(50, activation='relu'),
    Dense(1)
])
lstm_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lstm_model.fit(
    X_train_lstm, y_train_scaled,
    epochs=50, batch_size=64,
    validation_split=0.2, callbacks=[early_stop], verbose=1
)

# Holt-Winters Model
holt_model = ExponentialSmoothing(
    train_data['Close'], seasonal='add', seasonal_periods=24, trend='add'
).fit()

# Gradio Interface
def predict_price(input_date, input_time):
    from datetime import datetime

    # Parse input date and time
    input_datetime = datetime.strptime(f"{input_date} {input_time}", "%Y-%m-%d %H:%M:%S")

    # Generate features for prediction
    features_for_prediction = {
        'Open': train_data.iloc[-1]['Open'],
        'High': train_data.iloc[-1]['High'],
        'Low': train_data.iloc[-1]['Low'],
        'Volume': train_data.iloc[-1]['Volume'],
        'hour': input_datetime.hour,
        'day_of_week': input_datetime.dayofweek,
        'month': input_datetime.month,
        'Close_Lag_1': train_data.iloc[-1]['Close'],
        'Close_Lag_2': train_data.iloc[-1]['Close_Lag_1'],
        'Close_MA_3': (train_data.iloc[-1]['Close'] + train_data.iloc[-1]['Close_Lag_1'] +
                       train_data.iloc[-1]['Close_Lag_2']) / 3,
        'Close_MA_7': train_data['Close'].rolling(window=7).mean().iloc[-1],
        'Close_EMA_3': train_data.iloc[-1]['Close_EMA_3'],
        'Close_EMA_7': train_data.iloc[-1]['Close_EMA_7']
    }

    # Convert features to DataFrame
    features_df = pd.DataFrame([features_for_prediction])
    features_scaled = scaler_X.transform(features_df)

    # XGBoost Prediction
    xgb_prediction = best_xgb_model.predict(features_scaled)[0]

    # LSTM Prediction
    lstm_prediction = lstm_model.predict(features_scaled.reshape(1, 1, -1))[0, 0]
    lstm_prediction = scaler_y.inverse_transform([[lstm_prediction]])[0, 0]

    # Holt-Winters Prediction
    holt_prediction = holt_model.forecast(1)[0]

    # Combine predictions (weighted average)
    final_prediction = (xgb_prediction + lstm_prediction + holt_prediction) / 3

    return f"Predicted Closing Price: {final_prediction:.2f}"




<ipython-input-6-8e0f3de45147>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['hour'] = data.index.hour
<ipython-input-6-8e0f3de45147>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['day_of_week'] = data.index.dayofweek
<ipython-input-6-8e0f3de45147>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2053/2053 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0097 - val_loss: 5.4100e-04
Epoch 2/50
2053/2053 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 2.1062e-04 - val_loss: 6.4578e-04
Epoch 3/50
2053/2053 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 1.1472e-04 - val_loss: 6.8404e-04
Epoch 4/50
2053/2053 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - loss: 8.3736e-05 - val_loss: 5.8523e-04
Epoch 5/50
2053/2053 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - loss: 7.3543e-05 - val_loss: 9.5893e-04
Epoch 6/50
2053/2053 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step - loss: 5.8787e-05 - val_loss: 7.8051e-04
Epoch 7/50
2053/2053 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - loss: 5.4660e-05 - val_loss: 9.1550e-04
Epoch 8/50
2053/2053 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step - loss: 5.0790e-05 - val_loss: 0.0010
Epoch 9/50
2053/2053 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 5.0054e-05 - val_loss: 9.1275e-04
Epoch 10/50
2053/2053 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step - loss: 4.7842e-05 - val_loss: 8.5349e-04
Epoch 11/50
2053/2053 ━━━━━━━━━━━━━━━━━━━━ 13s 6

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/holtwinters/model.py:84: RuntimeWarning: overflow encountered in matmul
  return err.T @ err


In [14]:
import gradio as gr
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from tensorflow.keras.models import load_model
from statsmodels.tsa.holtwinters import ExponentialSmoothing

try:
    # Ensure these files exist in the directory
    xgb_model = best_xgb_model  # XGBoost model trained earlier
    lstm_model = lstm_model  # Trained LSTM model
    scaler_X = scaler_X  # Feature scaler
    scaler_y = scaler_y  # Target scaler
except Exception as e:
    print(f"Error loading models or scalers: {e}")
    raise


last_known_data = train_data.iloc[-1]

# Function to process input and make predictions
def predict_forex(input_date, input_time):
    try:
        # Combine input date and time
        input_datetime = pd.to_datetime(f"{input_date} {input_time}")

        # Generate features for prediction
        features_for_prediction = {
            'Open': last_known_data['Open'],  # Access from last known data
            'High': last_known_data['High'],
            'Low': last_known_data['Low'],
            'Volume': last_known_data['Volume'],
            'hour': input_datetime.hour,
            'day_of_week': input_datetime.dayofweek,
            'month': input_datetime.month,
            'Close_Lag_1': last_known_data['Close'],
            'Close_Lag_2': last_known_data['Close_Lag_1'],
            'Close_MA_3': (last_known_data['Close'] + last_known_data['Close_Lag_1'] + last_known_data['Close_Lag_2']) / 3,
            'Close_MA_7': (last_known_data['Close'] + last_known_data['Close_Lag_1'] + last_known_data['Close_Lag_2'] +
                           last_known_data.get('Close_Lag_3', last_known_data['Close']) +
                           last_known_data.get('Close_Lag_4', last_known_data['Close']) +
                           last_known_data.get('Close_Lag_5', last_known_data['Close']) +
                           last_known_data.get('Close_Lag_6', last_known_data['Close'])) / 7,
            'Close_EMA_3': last_known_data['Close_EMA_3'],
            'Close_EMA_7': last_known_data['Close_EMA_7']
        }

        # Create a DataFrame for scaling
        features_df = pd.DataFrame([features_for_prediction])

        # Scale features
        features_scaled = scaler_X.transform(features_df)

        # Make predictions using XGBoost
        rf_prediction = xgb_model.predict(features_scaled)[0]

        # Make predictions using LSTM
        features_scaled_reshaped = features_scaled.reshape(1, 1, -1)
        lstm_prediction = lstm_model.predict(features_scaled_reshaped)[0, 0]

        # Combine predictions (average)
        combined_prediction = (rf_prediction + lstm_prediction) / 2

        # Inverse scale the prediction
        final_prediction = scaler_y.inverse_transform([[combined_prediction]])[0, 0]

        # Holt-Winters prediction (for illustration, using seasonal period = 24)
        hw_model = ExponentialSmoothing(
            data['Close'], seasonal='add', seasonal_periods=24
        ).fit()
        hw_forecast = hw_model.forecast(1)[0]

        return f"XGB + LSTM Prediction: {final_prediction:.2f}, Holt-Winters: {hw_forecast:.2f}"
    except Exception as e:
        import traceback
        traceback.print_exc()    # Corrected indentation - this line caused the error
        # Return the error message
        return f"Error in prediction: {e}"

# Gradio interface
interface = gr.Interface(
    fn=predict_forex,
    inputs=[
        gr.components.Textbox(label="Date (YYYY-MM-DD)", placeholder="2025-01-27"),  # Change here
        gr.components.Textbox(label="Time (HH:MM:SS)", placeholder="17:00:00")  # Change here
    ],
    outputs=gr.components.Textbox(label="Predicted Closing Price")  # Change here
)

# Launch dashboard
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://42c04c176f94ecff88.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [1]:
!pip install pandas numpy scikit-learn xgboost tensorflow gradio statsmodels
!pip install --upgrade scikit-learn xgboost
!pip install pandas numpy tensorflow matplotlib
!pip install scikit-learn==1.3.0
!pip install xgboost

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import gradio as gr

# Load data
file_path = 'europeus.csv'
data = pd.read_csv(file_path, on_bad_lines='skip')

# Handling potential issues in the 'Time' column
try:
    data['Time'] = pd.to_datetime(data['Time'], errors='coerce')  # Handle errors by setting them to NaT
except pd.errors.ParserError as e:
    print(f"Error parsing 'Time' column: {e}")
    # Additional error handling if the 'Time' column format is unpredictable

# Remove rows with NaT (Not a Time) values in the 'Time' column
data = data.dropna(subset=['Time'])

# Set 'Time' as index
data.set_index('Time', inplace=True)

# Feature engineering
data['hour'] = data.index.hour
data['day_of_week'] = data.index.dayofweek
data['month'] = data.index.month
data['Close_Lag_1'] = data['Close'].shift(1)
data['Close_Lag_2'] = data['Close'].shift(2)
data['Close_MA_3'] = data['Close'].rolling(window=3).mean()
data['Close_MA_7'] = data['Close'].rolling(window=7).mean()
data['Close_EMA_3'] = data['Close'].ewm(span=3).mean()
data['Close_EMA_7'] = data['Close'].ewm(span=7).mean()
data = data.dropna()

# Split training and testing data
train_data = data[data.index < '2025-01-01']
test_data = data[data.index >= '2025-01-01']

# Features and target
features = ['Open', 'High', 'Low', 'Volume', 'hour', 'day_of_week', 'month',
            'Close_Lag_1', 'Close_Lag_2', 'Close_MA_3', 'Close_MA_7', 'Close_EMA_3', 'Close_EMA_7']
X_train = train_data[features]
y_train = train_data['Close']

# Scale features for models
scaler_X = StandardScaler()
scaler_y = StandardScaler() # Changed to StandardScaler
X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))



# XGBoost Model
from sklearn.exceptions import NotFittedError

# Ensure data is clean
if np.isnan(X_train_scaled).sum() > 0 or np.isinf(X_train_scaled).sum() > 0:
    print("Fixing NaNs/Infs in training data...")
    X_train.fillna(X_train.mean(), inplace=True)
    X_train_scaled = scaler_X.fit_transform(X_train)

# Validate hyperparameters
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1]
}

# Initialize XGBoost model with 'tree_method' set to 'hist' for CPU training
xgb_model = XGBRegressor(tree_method='hist')  # Force CPU training using 'hist'

try:
    # Fit XGBoost with RandomizedSearchCV
    xgb_random_search = RandomizedSearchCV(
        xgb_model, param_grid, n_iter=10, cv=3, scoring='neg_mean_squared_error', n_jobs=-1,
        random_state=42, error_score='raise'
    )
    xgb_random_search.fit(X_train_scaled, y_train)

    # Assign the best model
    best_xgb_model = xgb_random_search.best_estimator_

except ValueError as e:
    print(f"Error during model training: {e}")
    raise

# LSTM Model
X_train_lstm = X_train_scaled.reshape(X_train_scaled.shape[0], 1, X_train_scaled.shape[1])

lstm_model = Sequential([
    LSTM(100, activation='relu', input_shape=(1, X_train_scaled.shape[1]), return_sequences=True),
    Dropout(0.2),
    LSTM(50, activation='relu', return_sequences=False),
    Dropout(0.2),
    Dense(50, activation='relu'),
    Dense(1, activation='linear')  # Changed activation to 'linear'
])
lstm_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lstm_model.fit(
    X_train_lstm, y_train_scaled,
    epochs=50, batch_size=64,
    validation_split=0.2, callbacks=[early_stop], verbose=1
)


# Gradio Interface
last_known_data = train_data.iloc[-1]

def predict_forex(input_date, input_time):
    try:
        input_datetime_str = f"{input_date} {input_time}".replace("_", "-")
        input_datetime = pd.to_datetime(input_datetime_str)

        # Generate features for prediction
        features_for_prediction = {
            'Open': last_known_data['Open'],
            'High': last_known_data['High'],
            'Low': last_known_data['Low'],
            'Volume': last_known_data['Volume'],
            'hour': input_datetime.hour,
            'day_of_week': input_datetime.dayofweek,
            'month': input_datetime.month,
            'Close_Lag_1': last_known_data['Close'],
            'Close_Lag_2': last_known_data['Close_Lag_1'],
            'Close_MA_3': (last_known_data['Close'] + last_known_data['Close_Lag_1'] + last_known_data['Close_Lag_2']) / 3,
            'Close_MA_7': (last_known_data['Close'] + last_known_data['Close_Lag_1'] + last_known_data['Close_Lag_2'] +
                           last_known_data.get('Close_Lag_3', last_known_data['Close']) +
                           last_known_data.get('Close_Lag_4', last_known_data['Close']) +
                           last_known_data.get('Close_Lag_5', last_known_data['Close']) +
                           last_known_data.get('Close_Lag_6', last_known_data['Close'])) / 7,
            'Close_EMA_3': last_known_data['Close_EMA_3'],
            'Close_EMA_7': last_known_data['Close_EMA_7']
        }

        # Create a DataFrame for scaling
        features_df = pd.DataFrame([features_for_prediction])

        # Scale features

        # Scale features
        features_scaled = scaler_X.transform(features_df)

        # Make predictions using XGBoost
        rf_prediction = best_xgb_model.predict(features_scaled)[0]
        print(f"XGBoost Prediction: {rf_prediction}")  # Check prediction

        # Make predictions using LSTM
        features_scaled_reshaped = features_scaled.reshape(1, 1, -1)
        lstm_prediction = lstm_model.predict(features_scaled_reshaped)[0, 0]
        print(f"LSTM Prediction: {lstm_prediction}")  # Check prediction

        # Combine predictions (average)
        combined_prediction = (rf_prediction + lstm_prediction) / 2

        # Inverse scale the prediction
        final_prediction = scaler_y.inverse_transform([[combined_prediction]])[0, 0]
        # Holt-Winters prediction (for illustration, using seasonal period = 24)
        hw_model = ExponentialSmoothing(
            data['Close'], seasonal='add', seasonal_periods=24
        ).fit()
        hw_forecast = hw_model.forecast(1)[0]

        return f"XGB + LSTM Prediction: {final_prediction:.2f}, Holt-Winters: {hw_forecast:.2f}"
    except Exception as e:
        import traceback
        traceback.print_exc()
        return f"Error in prediction: {e}"

# Gradio interface
interface = gr.Interface(
    fn=predict_forex,
    inputs=[
        gr.components.Textbox(label="Date (YYYY-MM-DD)", placeholder="2025-01-27"),
        gr.components.Textbox(label="Time (HH:MM:SS)", placeholder="17:00:00")
    ],
    outputs=gr.components.Textbox(label="Predicted Closing Price")
)

# Launch dashboard
interface.launch()

  Using cached scikit_learn-1.6.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
Using cached scikit_learn-1.6.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.5 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.0
    Uninstalling scikit-learn-1.3.0:
      Successfully uninstalled scikit-learn-1.3.0
  Using cached scikit_learn-1.3.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
Using cached scikit_learn-1.3.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.9 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imbalanced-learn 0.13.0 requires scikit-learn<2,>=1

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2053/2053 ━━━━━━━━━━━━━━━━━━━━ 17s 7ms/step - loss: 0.0737 - val_loss: 0.0065
Epoch 2/50
2053/2053 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - loss: 0.0047 - val_loss: 0.0054
Epoch 3/50
2053/2053 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - loss: 0.0028 - val_loss: 0.0105
Epoch 4/50
2053/2053 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - loss: 0.0022 - val_loss: 0.0042
Epoch 5/50
2053/2053 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - loss: 0.0019 - val_loss: 0.0079
Epoch 6/50
2053/2053 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 0.0015 - val_loss: 0.0056
Epoch 7/50
2053/2053 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - loss: 0.0014 - val_loss: 0.0062
Epoch 8/50
2053/2053 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - loss: 0.0012 - val_loss: 0.0050
Epoch 9/50
2053/2053 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step - loss: 0.0012 - val_loss: 0.0032
Epoch 10/50
2053/2053 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - loss: 0.0010 - val_loss: 0.0067
Epoch 11/50
2053/2053 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 9.9805e-04 - val_loss: 0.0087
Epoch 12/50
2053/2053 ━━━━